# Mortes em Itália devido ao Covid-19, por região #

## Ligação ao Qgis ##

In [1]:
import os
import sys
from osgeo import ogr
os.environ['QT_QPA_PLATFORM'] = 'offscreen'
sys.path.insert(0,'/usr/local/share/qgis/python')
from qgis.core import *
QgsApplication.setPrefixPath("/usr/local", True) 
from qgis.gui import *
from qgis import processing

from qgis.PyQt.QtGui import QColor, QImage
from qgis.PyQt.QtCore import QSize, QBuffer, QIODevice

qgs = QgsApplication([], False)
qgs.initQgis()
print(QgsApplication.showSettings())

Application state:
QGIS_PREFIX_PATH env var:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS
Prefix:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS
Plugin Path:		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../PlugIns/qgis
Package Data Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources
Active Theme Name:	
Active Theme Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/resources/themes//icons/
Default Theme Path:	:/images/themes/default/
SVG Search Paths:	/usr/local/../Resources/svg/
		/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/svg/
		/Users/dianamartins/Library/Application Support/profiles/default/svg/
User DB Path:	/Users/dianamartins/anaconda3/QGIS.app/Contents/MacOS/../Resources/resources/qgis.db
Auth DB Path:	/Users/dianamartins/Library/Application Support/profiles/default/qgis-auth.db



## Carregar uma camada a partir de uma tabela guardada num Geopackage ##